In [58]:
import sqlite3
conn = sqlite3.connect('../../mismatch_db.db')

In [3]:
import pandas as pd
import os
import sys

def fasta2List(pathFasta):
    f = open(pathFasta, "r")
    title = []
    seq = []
    seq_temp = []
    for line in f:
        if line[0] == ">":
            seq.append(''.join(seq_temp).replace("\n", ""))
            title.append(line.replace("\n", ""))
            seq_temp = []
        else:
            seq_temp.append(line)
    seq.append(''.join(seq_temp).replace("\n", ""))
    seq.pop(0)
    dictionary = dict(zip(title, seq))
    return dictionary

In [ ]:
# Table mismatch
error_list = pd.read_csv("../../data/mismatch-analysis2/uniprot_errors_type3.txt", sep=" ", header=None)
human_exon_pos = pd.read_csv("../../data/mismatch-flagging/human_mismatch_exon_pos.tab", sep="\t")
primate_exon_pos = pd.read_csv("../../data/mismatch-analysis2/mismatch_exon_pos.tab", sep="\t")
# Script kirsley pr seq

In [5]:
# Table Protein
primate_prot = fasta2List("../../data/raw/uniprot-sequence/all_sequence.fasta")
human_prot = fasta2List("../../data/raw/uniprot-sequence/all_sequence.fasta")
primate_ensembl = pd.read_csv("../../data/mismatch-analysis2/transcript_ensembl_corrected2.tab", sep="\t")
human_ensembl = pd.read_csv("../../data/mismatch-flagging/human_transcript_ensembl_corrected2.tab", sep="\t")

In [52]:
primate_prot.update(human_prot)
ensembl_uniprot = pd.concat([human_ensembl, primate_ensembl])
ensembl_uniprot.rename(columns={'From':"prot_ID", 'To':"transcript_ID"}, inplace=True)
prot_name = []
orga = []
seq = []

for key, val in primate_prot.items():
    my_elem = key.split(" ")
    prot_name.append(my_elem[0][1:])
    orga.append([x for x in my_elem if "OX=" in x][0][3:])
    seq.append(val)
my_dict = {'prot_ID': prot_name, 'sequence': seq, 'organism': orga}
df = pd.DataFrame(my_dict)

In [60]:
prot_table = df.merge(ensembl_uniprot, on=["prot_ID"], how='left')

In [62]:
prot_table.to_sql(con=conn, name='protein', index=False, if_exists="append")

In [ ]:
# Table Transcript
primate_ensembl = pd.read_csv("../../data/mismatch-analysis2/transcript_ensembl_corrected2.tab", sep="\t")
human_ensembl = pd.read_csv("../../data/mismatch-flagging/transcript_ensembl_corrected2.tab", sep="\t")
primate_cds = fasta2List("../../data/mismatch-analysis2/CDS_all_filt.fasta")
human_cds = fasta2List("../../data/mismatch-flagging/human_CDS_all_corrected_filt.fasta")
primate_genom = fasta2List("../../data/mismatch-analysis2/genomic_all_filt.fasta")
human_genom = fasta2List("../../data/mismatch-flagging/human_genomic_all_corrected_filt.fasta")

In [ ]:
# Table exon_intron_map
# stage-thompson/data$ cat mismatch-analysis2/Exon_map.tab mismatch-analysis2/Intron_map.tab mismatch-flagging/human_Exon_map.tab mismatch-flagging/human_Intron_map.tab > full_exon_intron_map.tab
full_map = pd.read_csv("../../data/full_exon_intron_map.tab", sep="\t")

In [ ]:
# Table tblastn

In [ ]:
# Table mismatch_flag